## Assignment 2 

We are finding the best model to predict whether a person is cancer benign or malignant.

In [1]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
# set random seed to ensure that results are repeatable
np.random.seed(1)

## Load data 

In [3]:
# load data
cancer = pd.read_csv("breast-cancer-wisconsin.csv")

##  Conduct initial exploration of the data

In [4]:
# look at the data
cancer.head(3) # note that we don't want to dump all the data to the screen

,1000025,5,1,1.1,1.2,2,1.3,3,1.4,1.5,2.1
0,1002945,5,4,4,5,7,10,3,2,1,2
1,1015425,3,1,1,1,2,2,3,1,1,2
2,1016277,6,8,8,1,3,4,3,7,1,2


In [5]:
# generate a basic summary of the data
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   1000025  682 non-null    int64
 1   5        682 non-null    int64
 2   1        682 non-null    int64
 3   1.1      682 non-null    int64
 4   1.2      682 non-null    int64
 5   2        682 non-null    int64
 6   1.3      682 non-null    int64
 7   3        682 non-null    int64
 8   1.4      682 non-null    int64
 9   1.5      682 non-null    int64
 10  2.1      682 non-null    int64
dtypes: int64(11)
memory usage: 58.7 KB


In [6]:
# generate a statistical summary of the numeric value in the data
cancer.describe()

,1000025,5,1,1.1,1.2,2,1.3,3,1.4,1.5,2.1
count,6.820000e+02,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,1.076833e+06,4.441349,3.153959,3.218475,2.832845,3.236070,3.548387,3.445748,2.872434,1.604106,2.700880
std,6.210926e+05,2.822751,3.066285,2.989568,2.865805,2.224214,3.645226,2.451435,3.054065,1.733792,0.954916
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.774540e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171820e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238741e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [7]:
# Check the missing values by summing the total na's for each variable
cancer.isna().sum()

1000025    0
5          0
1          0
1.1        0
1.2        0
2          0
1.3        0
3          0
1.4        0
1.5        0
2.1        0
dtype: int64

There are no missing values

In [8]:
# create a list of these catagorical variables
category_var_list = list(cancer.select_dtypes(include='object').columns)
category_var_list

[]

### Summary the findings from our initial evaluation of the data

* there is no categorical variables and hence we can proceed 

#### Drop any columns/variables we will not be using

In [9]:
# Our target is price; but there are three related price variableds - price, price_gte_150, 
# and price_category. We need to drop price_gte_150, and price_category
cancer.drop(cancer.columns[0], axis=1, inplace=True)

In [10]:
# investigage how many rows remain 
cancer.shape

(682, 10)

#### Encode our categorical variables

since the "Class" variable only has two unique values (benign or malignant), there is no need to perform one-hot encoding on the target variable

###  Split data (train/test)

In [11]:
# Split the dataset into features (X) and target (y)
X = cancer.drop('2.1', axis=1)
y = cancer['2.1']

# Split the data into train and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)



## Save the data

In [12]:

train_X.to_csv('cancer_train_X.csv', index=False)
train_y.to_csv('cancer_train_y.csv', index=False)

test_X.to_csv('cancer_test_X.csv', index=False)
test_y.to_csv('cancer_test_y.csv', index=False)

In [13]:
# Check the class distribution of the target variable
class_counts = cancer['2.1'].value_counts()

# Print the class distribution
print("Class Distribution:")
print(class_counts)

Class Distribution:
2    443
4    239
Name: 2.1, dtype: int64


There is an imbalance in the dataset and hence we do oversampling 

In [14]:
from imblearn.over_sampling import RandomOverSampler

# Instantiate RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='minority')

# Apply random oversampling on training set
train_X,train_y = oversampler.fit_resample(train_X,train_y)

# Check the class distribution after random oversampling
train_y.value_counts()

4    356
2    356
Name: 2.1, dtype: int64

Now, train dataset is balanced and now we do undersampling for test dataset

In [15]:
from imblearn.under_sampling import RandomUnderSampler


# Apply random undersampling on test set
undersampler = RandomUnderSampler()
test_X, test_y = undersampler.fit_resample(test_X, test_y)


Now the dataset is balanced for both train and test sets.

# Logistic Regression using RandomSearch and Grid Search

In [16]:
score_measure = "accuracy"
kfolds = 5

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], # C is the regularization strength
              'penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'solver': ['saga', 'liblinear'],
              'max_iter': np.arange(500, 1000)}

lr = LogisticRegression()

rand_search = RandomizedSearchCV(estimator=lr, param_distributions=param_grid, cv=kfolds, n_iter=500,
                                 scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                                 return_train_score=True)

_ = rand_search.fit(train_X, train_y.astype('int'))

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestlr = rand_search.best_estimator_


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.9719196296661086
... with parameters: {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 636, 'C': 10}


C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
905 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
345 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 71, in _check_solver
    raise ValueError("penalty='none' is not supported for the liblinear sol

In [17]:
score_measure = "accuracy"
kfolds = 5
best_penality = rand_search.best_params_['penalty']
best_solver = rand_search.best_params_['solver']
min_regulization_strength=rand_search.best_params_['C']
min_iter = rand_search.best_params_['max_iter']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization_strength-0.1,min_regulization_strength+0.1, 0.01), 
    'penalty':[best_penality],
    'solver':[best_solver],
    'max_iter': np.arange(min_iter-200,min_iter+200, 50)
}

logreg =  LogisticRegression()
grid_search = GridSearchCV(estimator = logreg, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                return_train_score=True)

_ = grid_search.fit(train_X,train_y.astype(int))

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestlogreg = grid_search.best_estimator_


Fitting 5 folds for each of 160 candidates, totalling 800 fits
The best accuracy score is 0.9719196296661086
... with parameters: {'C': 9.9, 'max_iter': 436, 'penalty': 'l1', 'solver': 'liblinear'}


In [18]:
c_matrix = confusion_matrix(test_y,grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
performance = pd.concat([pd.DataFrame({'model':"Logistic using random & grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

Accuracy: 0.95
Precision: 0.9591836734693877
Recall: 0.94


,model,Accuracy,Precision,Recall,F1
0,Logistic using random & grid search,0.95,0.959184,0.94,0.949495


# SVM using RandomSearch and Grid Search

In [19]:
#Random search
score_measure = "accuracy"
kfolds = 5

param_grid = {'C':np.arange(0.1,50,10),  #  regularization parameter.
               'kernel':['linear', 'rbf','poly'],
              'gamma':['scale','auto'],
              'degree':np.arange(1,10), #degree is for the polynomial kernal
              'coef0':np.arange(1,10) #coef0 is for the polynomial kernal
                  
}

    

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestSVMrandom = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.9803309366689648
... with parameters: {'kernel': 'poly', 'gamma': 'scale', 'degree': 1, 'coef0': 2, 'C': 40.1}


In [20]:
#grid search 
score_measure = "accuracy"
kfolds = 3
best_kernel = rand_search.best_params_['kernel']
best_gamma = rand_search.best_params_['gamma']
min_regulization=rand_search.best_params_['C']
best_degree = rand_search.best_params_['degree']
best_coef0=rand_search.best_params_['coef0']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization-3,min_regulization+3), 
               'kernel':[best_kernel],
              'gamma':[best_gamma],
              'degree': np.arange(best_degree-1,best_degree+1),
            'coef0': np.arange(best_coef0-3,best_coef0+3)
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestSVMgrid = grid_search.best_estimator_

Fitting 3 folds for each of 72 candidates, totalling 216 fits
The best accuracy score is 0.9789325012705504
... with parameters: {'C': 37.1, 'coef0': -1, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}


In [21]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
performance = pd.concat([pd.DataFrame({'model':"SVM using random & grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

Accuracy=0.9600000 Precision=0.9600000 Recall=0.9600000 F1=0.9600000
Accuracy: 0.96
Precision: 0.96
Recall: 0.96


,model,Accuracy,Precision,Recall,F1
0,SVM using random & grid search,0.96,0.96,0.96,0.96


# Decision Tree classifier using RandomSearch and GridSearch

In [22]:
#decision tree classifier
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,42),  
    'min_samples_leaf': np.arange(1,42),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(1,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.9747168324633113
... with parameters: {'min_samples_split': 9, 'min_samples_leaf': 5, 'min_impurity_decrease': 0.0081, 'max_leaf_nodes': 34, 'max_depth': 9, 'criterion': 'entropy'}


In [23]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best accuracy score is 0.9676942775534325
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 36, 'min_impurity_decrease': 0.0009, 'min_samples_leaf': 5, 'min_samples_split': 3}


In [24]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
performance = pd.concat([pd.DataFrame({'model':"Decision using random & grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

Accuracy=0.9400000 Precision=0.9230769 Recall=0.9600000 F1=0.9411765
Accuracy: 0.94
Precision: 0.9230769230769231
Recall: 0.96


,model,Accuracy,Precision,Recall,F1
0,Decision using random & grid search,0.94,0.923077,0.96,0.941176


After creating the model, we identify that SVM is the best model as it has good recall score of 0.96 when compared with others and SVM model with poly kernel is able to predict whether a patient is having benign or malignant diesease as it has the highest accuracy.

Now, we try to introduce neural network model and compare which is the best.

# Neural Network Without Randomsearch and gridsearch cv

In [29]:
%%time
from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(hidden_layer_sizes=(60,50,40), solver='adam', max_iter=200)
_ = ann.fit(train_X,train_y)

Wall time: 545 ms


In [30]:
%%time
y_pred = ann.predict(test_X)

Wall time: 2.23 ms


In [31]:
from sklearn.metrics import classification_report
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           2       0.96      0.96      0.96        50
           4       0.96      0.96      0.96        50

    accuracy                           0.96       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.96      0.96      0.96       100



# With RandomizedSearchCV

In [32]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [500]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X,train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'solver': 'adam', 'max_iter': 500, 'learning_rate_init': 0.001, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (70, 50, 40), 'alpha': 0, 'activation': 'logistic'}
Wall time: 50.7 s


In [33]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           2       0.94      0.96      0.95        50
           4       0.96      0.94      0.95        50

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100

Wall time: 5 ms


# With GridSearchCV

In [34]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (50,), (70,), (90,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, .7, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [500]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
{'activation': 'tanh', 'alpha': 0.7, 'hidden_layer_sizes': (30,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'max_iter': 500, 'solver': 'adam'}
Wall time: 33.5 s


In [35]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           2       0.96      0.96      0.96        50
           4       0.96      0.96      0.96        50

    accuracy                           0.96       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.96      0.96      0.96       100

Wall time: 11 ms


In [36]:
c_matrix = confusion_matrix(test_y, y_pred)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
F1_lr=2*TP/(2*TP+FP+FN)


Accuracy=0.9600000 Precision=0.9600000 Recall=0.9600000 F1=0.9600000


In [37]:
performance = pd.concat([performance, pd.DataFrame({'model':"Neural network using random & grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Decision using random & grid search,0.94,0.923077,0.96,0.941176
0,Neural network using random & grid search,0.96,0.960000,0.96,0.960000


# Keras 

In [42]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from __future__ import print_function
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

# Keras using Randomisedgrid search 

In [91]:
%%time
import keras
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=9)),
    for hidden_layer_size in hidden_layer_sizes:
        ann.add(keras.layers.Dense(hidden_layer_size, kernel_initializer=tf.keras.initializers.GlorotUniform(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        ann.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    ann.compile(loss='binary_crossentropy', metrics=['accuracy'])
    return ann



Wall time: 0 ns


In [92]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=64,
    dropout = 0.0
)


In [93]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':["adam",'sgd']
}
keras_clf.get_params().keys()


dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [94]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(train_X, train_y, callbacks=callback, verbose=0)

2/2 [==============================] - 0s 5ms/step


In [95]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.001,
 'optimizer': 'adam',
 'model__hidden_layer_sizes': (70,),
 'model__dropout': 0,
 'epochs': 100,
 'batch_size': 60}

In [96]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.001, 'optimizer': 'adam', 'model__hidden_layer_sizes': (70,), 'model__dropout': 0, 'epochs': 100, 'batch_size': 60}


In [97]:
%%time
y_pred = best_net.predict(test_X)
print(classification_report(test_y, y_pred))

2/2 [==============================] - 0s 8ms/step
              precision    recall  f1-score   support

           2       0.96      0.96      0.96        50
           4       0.96      0.96      0.96        50

    accuracy                           0.96       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.96      0.96      0.96       100

Wall time: 82.5 ms


In [98]:
c_matrix = confusion_matrix(test_y, y_pred)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.9600000 Precision=0.9600000 Recall=0.9600000 F1=0.9600000


In [99]:
performance = pd.concat([performance, pd.DataFrame({'model':"deep neural network with randomisedgrid search using keras", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Decision using random & grid search,0.94,0.923077,0.96,0.941176
0,Neural network using random & grid search,0.96,0.960000,0.96,0.960000
0,deep neural network with randomisedgrid search...,0.00,NaN,0.00,0.000000
0,deep neural network with randomisedgrid search...,0.95,0.959184,0.94,0.949495
0,deep neural network with randomisedgrid search...,0.96,0.960000,0.96,0.960000


# Conclusion

In [100]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Model", "Accuracy","Precision","Recall","F1"]
table.add_row(["Logistic Regression using Random & Grid Search", "0.95","0.95","0.94","0.94"])
table.add_row(["SVM using Random & Grid Search","0.96","0.96","0.96","0.96"])
table.add_row(["Decision tree using Random & Grid Search", "0.94","0.92","0.96","0.94"])
table.add_row(["Neural network without Random & Grid search", "0.96","0.96","0.96","0.96"])
table.add_row(["Neural network with Random & Grid search", "0.96","0.96","0.96","0.96"])
table.add_row(["Keras for deep neural network with Randomised Grid search", "0.95","0.95","0.94","0.94"])

print(table)

+-----------------------------------------------------------+----------+-----------+--------+------+
|                           Model                           | Accuracy | Precision | Recall |  F1  |
+-----------------------------------------------------------+----------+-----------+--------+------+
|       Logistic Regression using Random & Grid Search      |   0.95   |    0.95   |  0.94  | 0.94 |
|               SVM using Random & Grid Search              |   0.96   |    0.96   |  0.96  | 0.96 |
|          Decision tree using Random & Grid Search         |   0.94   |    0.92   |  0.96  | 0.94 |
|        Neural network without Random & Grid search        |   0.96   |    0.96   |  0.96  | 0.96 |
|          Neural network with Random & Grid search         |   0.96   |    0.96   |  0.96  | 0.96 |
| Keras for deep neural network with Randomised Grid search |   0.95   |    0.95   |  0.94  | 0.94 |
+-----------------------------------------------------------+----------+-----------+-------

Based on the performance metrics provided in the table, the SVM model using Random & Grid Search appears to be the best model for the breast cancer Wisconsin dataset.

The SVM model has the highest accuracy score of 0.96, indicating that it correctly predicted benign or malignant cases in the dataset in 96% of the cases. Additionally, the model also has high precision, recall, and F1-score, all with values of 0.96, indicating that it has balanced performance in terms of both identifying true positive cases (precision) and capturing all positive cases (recall).

Comparing the SVM model with the other models in the table, it consistently performs well in all the metrics, outperforming the other models in terms of accuracy, precision, recall, and F1-score. The neural network models (both with and without Random & Grid search) also have the same accuracy and performance metrics as the SVM model, indicating that they are also competitive options. The decision tree model has slightly lower accuracy and F1-score compared to the SVM and neural network models, while the logistic regression model has lower precision and recall compared to the SVM model.